In [1]:
# ============================================================================
# PHASE 1: INSTALLATION AND SETUP
# ============================================================================

# Install required packages
!pip install convokit
!pip install transformers torch datasets accelerate
!pip install streamlit pyngrok
!pip install textblob vaderSentiment
!pip install streamlit-chat
!pip install --upgrade pip

import os
import re
import json
import random
import pandas as pd
import numpy as np
from collections import defaultdict
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from convokit import Corpus, download
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import streamlit as st
import time
from threading import Thread

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.2/206.2 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/convokit/utterance_simulator/unslothUtteranceSimulatorModel.py:2: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


Error from Unsloth: NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.
Using device: cpu


In [2]:
# ============================================================================
# PHASE 2: YODA DIALOGUE EXTRACTION FROM CORNELL CORPUS
# ============================================================================

class YodaDialogueExtractor:
    def __init__(self):
        self.corpus = None
        self.yoda_dialogues = []
        self.conversation_pairs = []

    def load_corpus(self):
        """Load Cornell Movie Corpus"""
        print("Downloading Cornell Movie Corpus...")
        corpus_path = download("movie-corpus")
        self.corpus = Corpus(filename=corpus_path)
        print("Corpus loaded successfully!")

    def extract_yoda_dialogues(self):
        """Extract Yoda's dialogues from the corpus"""
        print("Extracting Yoda dialogues...")

        yoda_utterances = []
        yoda_conversations = []

        # Search for Yoda-related characters and utterances
        yoda_indicators = [
            'yoda', 'master yoda', 'jedi master',
            # Look for characteristic Yoda speech patterns
            r'.*\b(strong|powerful|fear|anger|suffering|force|jedi)\b.*',
            r'.*you (are|will|must|should|can).*',
            r'.*(patience|meditate|train|destiny|chosen one).*'
        ]

        for conversation in self.corpus.iter_conversations():
            utterances = list(conversation.iter_utterances())

            for i, utterance in enumerate(utterances):
                speaker = utterance.speaker.id.lower() if utterance.speaker else ""
                text = utterance.text.lower()

                # Check if this is likely Yoda speaking
                is_yoda = False

                # Direct character name match
                if any(indicator in speaker for indicator in ['yoda']):
                    is_yoda = True

                # Check for Yoda speech patterns
                if not is_yoda:
                    yoda_patterns = [
                        r'\b(strong|powerful) .* you (are|will be)\b',
                        r'\b(fear|anger) .* (leads to|path to)\b',
                        r'\b(patience|train|learn) .* you must\b',
                        r'\bmuch to learn\b',
                        r'\bthe force\b.*\b(strong|with you)\b',
                        r'\b(do or do not|there is no try)\b',
                        r'\b(judge me by my size)\b',
                        r'\byoung (padawan|skywalker|one)\b'
                    ]

                    if any(re.search(pattern, text) for pattern in yoda_patterns):
                        is_yoda = True

                if is_yoda:
                    yoda_utterances.append({
                        'text': utterance.text,
                        'speaker': speaker,
                        'conversation_id': conversation.id
                    })

                    # Get context (previous utterance as input)
                    if i > 0:
                        prev_utterance = utterances[i-1]
                        self.conversation_pairs.append({
                            'input': prev_utterance.text,
                            'output': utterance.text,
                            'context': 'movie_dialogue'
                        })

        self.yoda_dialogues = yoda_utterances
        print(f"Found {len(yoda_utterances)} Yoda utterances")
        print(f"Created {len(self.conversation_pairs)} conversation pairs")

        return self.yoda_dialogues, self.conversation_pairs

    def enhance_with_synthetic_data(self):
        """Add synthetic Yoda-style introspective dialogues"""
        synthetic_pairs = [
            {"input": "I feel lost and don't know what to do", "output": "Lost, feel you do? Guide you, the Force will. Trust in yourself, you must. Clear, your path will become."},
            {"input": "I'm afraid of failing", "output": "Fear of failure, sense in you I do. But fail, sometimes we must, to learn we do. Stronger, failure makes you."},
            {"input": "How do I find inner peace?", "output": "Inner peace, from within it comes. Quiet your mind, you must. Feel the Force flowing through you. Meditate, daily you should."},
            {"input": "I feel angry about everything", "output": "Anger, to the dark side it leads. Breathe deeply, you must. Let go of your anger. Understanding, the path to peace it is."},
            {"input": "What is my purpose in life?", "output": "Your purpose, already know it you do. Deep within, listen you must. Help others, a noble path it is. Follow your heart, you should."},
            {"input": "I don't believe in myself", "output": "Believe in yourself, essential it is. The Force, strong with you it is. Doubt, your enemy it becomes. Trust your abilities, you must."},
            {"input": "I feel overwhelmed by problems", "output": "Overwhelmed, natural it feels. But one problem at a time, solve you must. Patience, your greatest ally it is. Focus on the present, you should."},
            {"input": "How do I handle difficult people?", "output": "Difficult people, teachers they are. Patience with them, practice you must. Compassion, even for enemies, show you should. Their pain, understand you must."},
            {"input": "I made a terrible mistake", "output": "Mistakes, the greatest teacher they are. Learn from this, you must. Perfect, no one is. Grow stronger, from failures you will."},
            {"input": "I feel sad and depressed", "output": "Sadness, feel it you must. But consumed by it, be you should not. Pass, all emotions do. Hope, always there is."},
            {"input": "What are my core values?", "output": "Your values, deep within they lie. Reflect on what matters, you must. Guide your actions, they should. Honor them, you must."},
            {"input": "How do I make tough decisions?", "output": "Decisions, trust your feelings you must. The Force, guide you it will. Clear your mind of doubt. Listen to your heart, you should."},
            {"input": "I feel like giving up", "output": "Give up, a Jedi never does. Difficult, the path may be. But continue, you must. Stronger than you know, you are."},
            {"input": "How do I deal with change?", "output": "Change, constant it is. Resist it, suffer you will. Embrace change, flow with it you must. Opportunity, in change there is."},
            {"input": "I worry too much about the future", "output": "The future, always in motion it is. Control it, you cannot. Present moment, your power lies in. Worry, clouds your judgment it does."}
        ]

        self.conversation_pairs.extend(synthetic_pairs)
        print(f"Added {len(synthetic_pairs)} synthetic conversation pairs")
        print(f"Total conversation pairs: {len(self.conversation_pairs)}")

        return self.conversation_pairs

# Initialize and run extraction
extractor = YodaDialogueExtractor()
extractor.load_corpus()
yoda_dialogues, conversation_pairs = extractor.extract_yoda_dialogues()
enhanced_pairs = extractor.enhance_with_synthetic_data()

# Display samples
print("\nSample Yoda dialogues found:")
for i, dialogue in enumerate(yoda_dialogues[:5]):
    print(f"{i+1}. {dialogue['text']}")

print("\nSample conversation pairs:")
for i, pair in enumerate(conversation_pairs[:5]):
    print(f"{i+1}. Input: {pair['input']}")
    print(f"   Output: {pair['output']}")
    print("---")


No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem
Corpus loaded successfully!
Extracting Yoda dialogues...
Found 25 Yoda utterances
Created 14 conversation pairs
Added 15 synthetic conversation pairs
Total conversation pairs: 29

Sample Yoda dialogues found:
1. Oh, you know what we do or do not need to know? Since when have you become a media expert?
2. That man up there lived in similar times.  He had chaos and civil war and hatred between the races ... Sometimes I go to the Lincoln Room at the White House and just pray.  You know, the liberals act like idealism belongs to them, but it's not true.  My family went Republican because Lincoln freed the slaves.  My grandmother was an abolitionist.  It was Quakers who founded Whittier, my hometown, to abolish slavery.  They were conservative Bible fol

In [3]:
# ============================================================================
# PHASE 3: MODEL TRAINING
# ============================================================================

class YodaChatbotTrainer:
    def __init__(self, model_name="microsoft/DialoGPT-small"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None

    def load_model_and_tokenizer(self):
        """Load pre-trained model and tokenizer"""
        print(f"Loading model: {self.model_name}")

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Add padding token if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(self.model_name)
        self.model.to(device)

        print("Model and tokenizer loaded successfully!")

    def prepare_dataset(self, conversation_pairs):
        """Prepare dataset for training"""
        training_texts = []

        for pair in conversation_pairs:
            # Format as conversation
            text = f"<|startoftext|>Human: {pair['input']}<|sep|>Yoda: {pair['output']}<|endoftext|>"
            training_texts.append({"text": text})

        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                padding=True,
                max_length=512
            )

        # Create dataset
        dataset = Dataset.from_list(training_texts)
        tokenized_dataset = dataset.map(tokenize_function, batched=True)

        return tokenized_dataset

    def train_model(self, conversation_pairs, output_dir="./yoda-chatbot-cornell"):
        """Train the model"""
        print("Preparing dataset for training...")
        tokenized_dataset = self.prepare_dataset(conversation_pairs)

        # Data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
        )

        # Training arguments - optimized for Colab
        training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=3,
            per_device_train_batch_size=2,  # Small batch size for Colab
            gradient_accumulation_steps=4,   # Effective batch size = 2*4 = 8
            warmup_steps=100,
            logging_steps=50,
            save_steps=500,
            save_strategy="epoch",
            dataloader_num_workers=0,
            fp16=torch.cuda.is_available(),
            dataloader_pin_memory=False,
            report_to=None,  # Disable wandb
        )

        # Initialize trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_dataset,
            tokenizer=self.tokenizer,
        )

        print("Starting training...")
        trainer.train()

        # Save the model
        trainer.save_model(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        print(f"Model trained and saved to {output_dir}")
        return output_dir

# Initialize trainer
trainer = YodaChatbotTrainer()
trainer.load_model_and_tokenizer()

# Train the model
print("Starting model training with Cornell Movie Corpus data...")
model_path = trainer.train_model(enhanced_pairs)

Loading model: microsoft/DialoGPT-small


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chinmayi5405 (chinmayi5405-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Model trained and saved to ./yoda-chatbot-cornell


In [10]:
# ============================================================================
# PHASE 4: CHATBOT LOGIC WITH EMOTION DETECTION
# ============================================================================

class EmotionDetector:
    def __init__(self):
        self.sentiment_analyzer = SentimentIntensityAnalyzer()

    def detect_emotion(self, text):
        """Detect emotion in user input"""
        sentiment_scores = self.sentiment_analyzer.polarity_scores(text)

        # Simple emotion mapping
        compound = sentiment_scores['compound']

        if compound >= 0.5:
            emotion = 'very_positive'
        elif compound >= 0.1:
            emotion = 'positive'
        elif compound <= -0.5:
            emotion = 'very_negative'
        elif compound <= -0.1:
            emotion = 'negative'
        else:
            emotion = 'neutral'

        return {
            'emotion': emotion,
            'compound_score': compound,
            'scores': sentiment_scores
        }

class YodaChatbot:
    def __init__(self, model_path="./yoda-chatbot-cornell"):
        self.model_path = model_path
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(model_path)
        self.model.to(device)
        self.emotion_detector = EmotionDetector()

        print("Yoda Bot loaded successfully!")

    def generate_response(self, user_input, max_length=100):
        """Generate Yoda response"""
        # Detect emotion
        emotion_data = self.emotion_detector.detect_emotion(user_input)

        # Format input
        prompt = f"<|startoftext|>Human: {user_input}<|sep|>Yoda:"

        # Tokenize
        inputs = self.tokenizer.encode(prompt, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=len(inputs[0]) + max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.encode("<|endoftext|>")[0]
            )

        # Decode response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just Yoda's response
        yoda_response = full_response.split("Yoda:")[-1].strip()
        if "<|endoftext|>" in yoda_response:
            yoda_response = yoda_response.split("<|endoftext|>")[0].strip()

        return yoda_response, emotion_data

# Initialize chatbot
print("Initializing Yoda Bot...")
yoda_bot = YodaChatbot(model_path)

print("Setup complete! Ready to create Streamlit interface...")

Initializing Yoda Bot...
Yoda Bot loaded successfully!
Setup complete! Ready to create Streamlit interface...


In [9]:
from google.colab import files
uploaded = files.upload()


Saving image-removebg-preview (22).png to image-removebg-preview (22).png


In [11]:
# ============================================================================
# PHASE 5: STREAMLIT UI
# ============================================================================

# Save the Streamlit app code to a file (escape triple quotes properly)
streamlit_app_code = """
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import random
import os
import base64

# Page config
st.set_page_config(
    page_title="Yoda Bot - Jedi Wisdom Chatbot",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for styling
st.markdown('''<style>
    .main {
        background: linear-gradient(135deg, #0c0c0c 0%, #1a1a2e 50%, #16213e 100%);
    }
    .stTextInput > div > div > input {
        background-color: rgba(255, 255, 255, 0.1);
        color: #00ff00;
        border: 2px solid #00ff00;
        border-radius: 10px;
    }
    .chat-message {
        padding: 1rem;
        border-radius: 10px;
        margin-bottom: 1rem;
    }
    .user-message {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        margin-left: 20%;
    }
    .yoda-message {
        background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        color: white;
        margin-right: 20%;
    }
    .yoda-avatar {
        text-align: center;
        font-size: 4rem;
        margin: 1rem;
        animation: pulse 2s infinite;
    }
    @keyframes pulse {
        0% { transform: scale(1); }
        50% { transform: scale(1.1); }
        100% { transform: scale(1); }
    }
    .emotion-indicator {
        padding: 0.5rem;
        border-radius: 5px;
        margin: 0.5rem 0;
        text-align: center;
        font-weight: bold;
    }
    .emotion-positive { background-color: #4CAF50; color: white; }
    .emotion-negative { background-color: #f44336; color: white; }
    .emotion-neutral { background-color: #2196F3; color: white; }
</style>''', unsafe_allow_html=True)

# Load model and components (cached)
@st.cache_resource
def load_yoda_components():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_path = "./yoda-chatbot-cornell"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
    model.to(device)
    sentiment_analyzer = SentimentIntensityAnalyzer()
    return model, tokenizer, sentiment_analyzer, device

# Emotion detection function
def detect_emotion(text, sentiment_analyzer):
    scores = sentiment_analyzer.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.1:
        return 'positive', compound
    elif compound <= -0.1:
        return 'negative', compound
    else:
        return 'neutral', compound

# Yoda rephraser
import re

def rephrase_yoda(sentence):
    sentence = sentence.strip()
    if not sentence:
        return ""

    # Basic clause split
    parts = re.split(r'[\.\?!]', sentence)[0].split()
    if len(parts) < 4:
        return sentence + ", yes."

    # Find subject (assume 1st word), verb (assume 2nd or 3rd)
    # Move object or remainder to front
    subject = parts[0]
    verb = parts[1] if len(parts) > 2 else ""
    rest = parts[2:]

    # Construct Yoda-style
    yoda_sentence = f"{' '.join(rest)}, {subject} {verb}".strip().capitalize() + ", hmmm."
    return yoda_sentence


# Response generation function
def generate_yoda_response(user_input, model, tokenizer, device):
    prompt = f"<|startoftext|>Human: {user_input}<|sep|>Yoda:"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=len(inputs[0]) + 80,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.encode("<|endoftext|>")[0]
        )
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    yoda_response = full_response.split("Yoda:")[-1].strip()
    if "<|endoftext|>" in yoda_response:
        yoda_response = yoda_response.split("<|endoftext|>")[0].strip()
    return yoda_response

# Session state
if 'messages' not in st.session_state:
    st.session_state.messages = []
if 'emotions' not in st.session_state:
    st.session_state.emotions = []

# Load components
try:
    model, tokenizer, sentiment_analyzer, device = load_yoda_components()
    model_loaded = True
except:
    model_loaded = False
    st.error("Model not found. Please run the training script first.")

# Title
st.title("Yoda Bot - Jedi Wisdom Chatbot")

# Show image if uploaded
image_path = "yoda.png"
if os.path.exists(image_path):
    st.image(image_path, width=200, caption="Master Yoda", use_column_width=False)
else:
    st.warning("Yoda image not found. Please upload yoda.png.")

st.markdown('<div class="yoda-avatar">🧙</div>', unsafe_allow_html=True)
st.markdown('''
<div style="text-align: center; color: #00ff00; font-size: 1.2rem; margin-bottom: 2rem;">
    "Judge me by my size, do you? And well you should not. For my ally is the Force, and a powerful ally it is."
</div>
''', unsafe_allow_html=True)

# Sidebar
with st.sidebar:
    st.header("About Master Yoda")
    st.write("This chatbot channels the wisdom of Master Yoda to help you with: Self-reflection and introspection, Emotional guidance, Life decisions")
    st.header("Emotional State")
    if st.session_state.emotions:
        latest_emotion = st.session_state.emotions[-1]
        emotion_class = f"emotion-{latest_emotion[0]}"
        st.markdown(f'<div class="{emotion_class}">{latest_emotion[0].title()} ({latest_emotion[1]:.2f})</div>', unsafe_allow_html=True)

# Display chat history
for i, (msg, emotion) in enumerate(zip(st.session_state.messages, st.session_state.emotions)):
    role = "user-message" if i % 2 == 0 else "yoda-message"
    speaker = "You" if i % 2 == 0 else "Master Yoda"
    st.markdown(f'''
    <div class="chat-message {role}">
        <strong>{speaker}:</strong> {msg}
    </div>
    ''', unsafe_allow_html=True)

# Handle message
def handle_submit():
    user_input = st.session_state.get("user_input", "")
    if user_input and model_loaded:
        st.session_state.messages.append(user_input)
        emotion, score = detect_emotion(user_input, sentiment_analyzer)
        st.session_state.emotions.append((emotion, score))
        with st.spinner("Master Yoda is consulting the Force..."):
            time.sleep(1)
            raw_reply = generate_yoda_response(user_input, model, tokenizer, device)
            yoda_reply = rephrase_yoda(raw_reply)
        st.session_state.messages.append(yoda_reply)
        st.session_state.emotions.append(('neutral', 0))
        st.session_state.user_input = ""  # Clear input field (safe here in callback)

# Input
col1, col2, col3 = st.columns([1, 1, 1])
with col2:
    st.text_input(
        "Speak with Master Yoda...",
        key="user_input",
        placeholder="Young padawan, what troubles you?",
        on_change=handle_submit
    )

# Footer
st.markdown('<div style="text-align: center; margin-top: 3rem; color: #666; font-style: italic;">"Do or do not, there is no try." - Master Yoda</div>', unsafe_allow_html=True)
"""

# Save app
with open("yoda_chatbot_app.py", "w") as f:
    f.write(streamlit_app_code)

print("✅ Streamlit app saved as 'yoda_chatbot_app.py'")
print("➡️ To run it in Colab, use:")
print("!streamlit run yoda_chatbot_app.py & npx localtunnel --port 8501")


✅ Streamlit app saved as 'yoda_chatbot_app.py'
➡️ To run it in Colab, use:
!streamlit run yoda_chatbot_app.py & npx localtunnel --port 8501


In [12]:
!ngrok config add-authtoken 2jIGiWX8itjxErTlTDklcjaNnXC_51yoEom7hGoG7VFdX2DQq

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:


import subprocess
import time
import threading
from pyngrok import ngrok
import requests

def run_streamlit():
    """Run Streamlit app in background"""
    subprocess.run(["streamlit", "run", "yoda_chatbot_app.py", "--server.port=8501", "--server.headless=true"],
                   capture_output=True)

def setup_ngrok_tunnel():
    """Setup ngrok tunnel for public access"""
    print("Setting up ngrok tunnel...")

    # Kill any existing ngrok processes
    ngrok.kill()

    # Start ngrok tunnel
    public_url = ngrok.connect(8501)
    print(f"\\n Your Yoda Chatbot is live at: {public_url}")
    print("\\n Share this URL with anyone to let them chat with Master Yoda!")

    return public_url

def main():
    print(" LAUNCHING YODA BOT...")

    # Start Streamlit in background thread
    streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
    streamlit_thread.start()

    # Wait for Streamlit to start
    print("Waiting for Streamlit to start...")
    time.sleep(10)

    # Check if Streamlit is running
    try:
        response = requests.get("http://localhost:8501")
        if response.status_code == 200:
            print(" Streamlit is running successfully!")
        else:
            print(" Streamlit might not be fully ready yet...")
    except:
        print(" Streamlit connection check failed, but proceeding...")

    # Setup ngrok tunnel
    try:
        public_url = setup_ngrok_tunnel()

        print("\\n" + "="*60)
        print(" YODA CHATBOT IS NOW LIVE!")
        print("="*60)
        print(f" Public URL: {public_url}")
        print(" Local URL: http://localhost:8501")

        print("="*60)

        # Keep the tunnel alive
        print("\\nKeeping the tunnel alive... Press Ctrl+C to stop")
        try:
            while True:
                time.sleep(60)
                print("Chatbot is still running...")
        except KeyboardInterrupt:
            print("\\n Shutting down...")
            ngrok.kill()

    except Exception as e:
        print(f" Error setting up ngrok: {e}")
        print(" You can still access the app locally at: http://localhost:8501")

if __name__ == "__main__":
    main()

 LAUNCHING YODA BOT...
Waiting for Streamlit to start...
 Streamlit is running successfully!
Setting up ngrok tunnel...
\n Your Yoda Chatbot is live at: NgrokTunnel: "https://091134027e8f.ngrok-free.app" -> "http://localhost:8501"
\n Share this URL with anyone to let them chat with Master Yoda!
\n============================================================
 YODA CHATBOT IS NOW LIVE!
 Public URL: NgrokTunnel: "https://091134027e8f.ngrok-free.app" -> "http://localhost:8501"
 Local URL: http://localhost:8501
\nKeeping the tunnel alive... Press Ctrl+C to stop
Chatbot is still running...
Chatbot is still running...
Chatbot is still running...
Chatbot is still running...
Chatbot is still running...
Chatbot is still running...
\n Shutting down...


In [ ]:
!zip -r /content/yoda-chatbot-cornell.zip /content/yoda-chatbot-cornell
!zip -r /content/wandb.zip /content/wandb

  adding: content/yoda-chatbot-cornell/ (stored 0%)
  adding: content/yoda-chatbot-cornell/tokenizer.json (deflated 82%)
  adding: content/yoda-chatbot-cornell/model.safetensors